In [1]:
using Distributions
using Roots

In [2]:
N = 1000
R = 100
S = N * R
T = 100
# 大切なのはNのオーダーに対するalphaのオーダー
# N = 1000でalpha = 0.001より大きいと結果はほぼ0.5
alpha = -0.001
a = 2
b = 2;

上で述べたこと諸々含めて、1000人ゲーム、alpha = -0.001としてシミュレーションデータを出す。

In [3]:
function f(alpha, N, u_t)
    function f1(equi)
        equi - sum([1/(1 + exp(i + alpha*N*(1-2*equi))) for i in u_t])/N
    end
end

f (generic function with 1 method)

In [5]:
result = zeros(T)
srand(123)
beta_dis = Beta(a, b)  
u = rand(beta_dis, N*T)
u = reshape(u, (N, T)) - 0.5
for t in 1:T
    u_t = u[:, t]
    result[t] = fzero(f(alpha, N, u_t), 10e-5, 1 - 10e-5)
end
result

100-element Array{Float64,1}:
 0.502454
 0.499006
 0.499987
 0.500211
 0.501364
 0.503017
 0.500233
 0.498578
 0.499851
 0.498547
 0.500707
 0.501935
 0.498771
 ⋮       
 0.502113
 0.499905
 0.498734
 0.499551
 0.501096
 0.501458
 0.500891
 0.497682
 0.498371
 0.497559
 0.498556
 0.498468

推定

In [6]:
srand(1234)
beta_dis = Beta(a, b)
u = rand(beta_dis, S) - 0.5;

In [13]:
function object(result, u, a, b, alpha)
    be = Beta(a, b)
    (sum([((i + 0.5)^(a - 2))*((0.5 - i)^(b - 2))*sum([1/(1 + exp(i + alpha*N*(1 - 2*j))) for j in result])/T for i in u])/(S*6*beta(a, b)) - cdf(be, 0.5))^2
end

object (generic function with 1 method)

In [9]:
w = rand(beta_dis, 10) - 0.5;

In [10]:
sum([((i + 0.5)^(a - 2))*((0.5 - i)^(b - 2))*sum([1/(1 + exp(i + alpha*N*(1 - 2*j))) for j in result])/T for i in w])/S

4.931507189634179e-5

In [16]:
# 真の値を入れれば極めて小さくなる。
object(result, u, a, b, alpha)

3.804121601295839e-9

こいつを最小化する

In [17]:
using Optim

In [18]:
using JuMP

LoadError: [91mArgumentError: Module JuMP not found in current path.
Run `Pkg.add("JuMP")` to install the JuMP package.[39m